## Python Script to transform data

The script focuses on bring RECS data from 2001, 2005, 2009 and 2015 into 
1. common format
2. columns mapped correctly to the correct fields
3. calaculations done as required
4. elimination of columns not needed

### Rules:
1.	All columns starting with "Z" is dropped
2.	YearMade is converted to a range as per 2015 standards
3.	Drop 1997
4.	Club all electronics into 3 categories TVREL, PCOFFEQUIP, PHONE
5.	All comsumption will be BTU and KWH will not be used
6.	Calucaled Fields
    - "2001 - TOTROOMS - sum(BEDROOMS,NCOMBATH,NHAFBATH,OTHROOMS)"
	- TOTALBTU = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
	- TOTALBTUSPH = BTULPSPH + BTUNGSPH + BTUFOSPH + BTUELSPH
	- TOTALBTUWTH = BTULPWTH + BTUNGWTH + BTUFOWTH + BTUELWTH
	- TOTALBTUOTH = BTULPAPL + BTUNGOTH + BTUFOAPL + BTUELOTH + BTUELFRG +BTUELCOL
	- TOTALDOLLAR = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
	- TOTALDOLSPH = DOLLPSPH+ DOLNGSPH + DOLFOSPH + DOLELSPH
	- TOTALDOLWTH = DOLLPWTH+ DOLNGWTH+ DOLFOWTH+ DOLELWTH
	- TOTALDOLOTH = DOLLPOTH + DOLNGOTH+ DOLFOOTH + DOLELOTH + DOLELFRG + DOLELCOL


In [1]:
# import all dependecies
import pandas as pd
import os
import csv

In [65]:
# Global Notebook Variables and Initialization
dataFilePath = "dataforfinalproject/RawDataFiles"

codebook_path = "dataforfinalproject/RawDataFiles/Codebooks"

years = [2001, 2005, 2009, 2015]

dfs = {}

dataFiles = ['target_final_2001.csv', "RECS05alldata.csv", "recs2009_public.csv", "recs2015_public_v4.csv"]

col_list = ['2001_requiredCols.txt', '2005_requiredCols.txt', '2009_requiredCols.txt', '2015_requiredCols.txt']

In [66]:
# read the files into a dataframe and compress dataframe to have only required cols
for i, y in enumerate(years):
    print(y)
    df = pd.read_csv(os.path.join(dataFilePath, dataFiles[i]), low_memory=False)
    l = pd.read_csv(os.path.join(codebook_path, col_list[i]), header= None, names = ['cols']).cols.tolist()
    dfs[y] = df[l]

2001
2005
2009
2015


### Format and tranform 2001 data

In [67]:
# change columnname YearMade to YearMadeRange
dfs[2015].rename(columns = {"YEARMADERANGE":"YEARMADE"}, inplace = True)

# change columnname Basefin to Cellar
dfs[2015].rename(columns = {"BASEFIN":"CELLAR"}, inplace = True)
#dfs[2015].YEARMADE.head()

# change OCCUPYYRANGE to OCCUPYY
dfs[2015].rename(columns = {"OCCUPYYRANGE":"OCCUPYY"}, inplace = True)
dfs[2009].rename(columns = {"OCCUPYYRANGE":"OCCUPYY"}, inplace = True)

# change USENG to UGASHERE
dfs[2009].rename(columns = {"USENG":"UGASHERE"}, inplace = True)

# change "STOVENFU" to "STOVENFUEL"
dfs[2001].rename(columns = {"STOVENFU":"STOVENFUEL"}, inplace = True)
dfs[2005].rename(columns = {"STOVENFU":"STOVENFUEL"}, inplace = True)

# rename columns OUTLGTNT(2001), NOUTLGTNT(2005, 2009) to LGTOUTNUM
dfs[2001].rename(columns = {"OUTLGTNT":"LGTOUTNUM"}, inplace = True)
dfs[2005].rename(columns = {"NOUTLGTNT":"LGTOUTNUM"}, inplace = True)
dfs[2009].rename(columns = {"NOUTLGTNT":"LGTOUTNUM"}, inplace = True)

# rename ORIGIN1 to Householder_Race in 2001 and 2005
dfs[2001].rename(columns = {"ORIGIN1":"HOUSEHOLDER_RACE"}, inplace = True)
dfs[2005].rename(columns = {"ORIGIN1":"HOUSEHOLDER_RACE"}, inplace = True)
dfs[2009].rename(columns = {"Householder_Race":"HOUSEHOLDER_RACE"}, inplace = True)

# rename CD65 and HD65 to CDD65 and HDD65
dfs[2001].rename(columns = {"CD65":"CDD65", "HD65":"HDD65"}, inplace = True)

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [68]:
# Calculate totrooms for 2001 data
dfs[2001]['TOTROOMS'] = dfs[2001].BEDROOMS + dfs[2001].NCOMBATH  + dfs[2001].NHAFBATH +dfs[2001].OTHROOMS


In [69]:
equipAux_type = ["REVERSE","WARMAIR","STEAMR","PERMELEC","PIPELESS","ROOMHEAT","WOODKILN","CARRYEL","CARRYKER",
                 "CHIMNEY","RANGE","DIFEQUIP"]
for e in equipAux_type:
    dfs[2015][e] = 0
    
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 1,"CARRYEL"] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 2,'WOODKILN'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 3,'PIPELESS'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 4,'CHIMNEY'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 9,'DIFEQUIP'] = 1

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [70]:
# For EquipAux types in 2001 to 2009, if value is -2 change it to 0
for e in equipAux_type:
    dfs[2001].loc[dfs[2001][e] == -2,e] = 0
    dfs[2005].loc[dfs[2005][e] == -2,e] = 0
    dfs[2009].loc[dfs[2009][e] == -2,e] = 0   


In [71]:
equipAux_fuel = ["ELECAUX","UGASAUX","LPGAUX","FOKRAUX","WOODAUX","OTHERAUX"]

for e in equipAux_fuel:
    dfs[2015][e] = 0
    dfs[2009][e] = 0
    

#2015 changes
dfs[2015].loc[dfs[2015].FUELAUX == 1,"UGASAUX"] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 2,'LPGAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 3,'FOKRAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 5,'ELECAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 7,'WOODAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 21,'OTHERAUX'] = 1




E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [72]:
# changes to 2001 to 2005
# combine the value for FO and Kero to one column and Solar into Other columns

dfs[2001]['FOKRAUX'] = 0
dfs[2005]['FOKRAUX'] = 0


dfs[2001].loc[(dfs[2001]['FOILAUX'] == 1) | (dfs[2001]['KEROAUX'] == 1),"FOKRAUX"] = 1
dfs[2005].loc[(dfs[2005]['FOILAUX'] == 1) | (dfs[2005]['KEROAUX'] == 1),"FOKRAUX"] = 1

dfs[2001].loc[(dfs[2001]['SOLARAUX'] == 1) | (dfs[2001]['OTHERAUX'] == 1),"OTHERAUX"] = 1
dfs[2005].loc[(dfs[2005]['SOLARAUX'] == 1) | (dfs[2005]['OTHERAUX'] == 1),"OTHERAUX"] = 1


In [73]:
for e in equipAux_fuel:
    dfs[2001].loc[dfs[2001][e] == -2,e] = 0
    dfs[2005].loc[dfs[2005][e] == -2,e] = 0   


In [74]:
# transform 2009 Auxillary fuel type to standard fuel types as specified in years (2001, 2005 and 2015)
equipAux_fuel_2009 = ["FURNFUEL","RADFUEL","PIPEFUEL","RMHTFUEL","HSFUEL","FPFUEL","RNGFUEL","DIFFUEL"]
for e in equipAux_fuel_2009:
    dfs[2009].loc[dfs[2009][e] == 1,"UGASAUX"] = 1
    dfs[2009].loc[dfs[2009][e] == 2,'LPGAUX'] = 1
    dfs[2009].loc[(dfs[2009][e] == 3) | (dfs[2009][e] == 4) ,'FOKRAUX'] = 1
    dfs[2009].loc[dfs[2009][e] == 5,'ELECAUX'] = 1
    dfs[2009].loc[dfs[2009][e] == 7,'WOODAUX'] = 1
    dfs[2009].loc[dfs[2009][e] > 7,'OTHERAUX'] = 1
    

In [75]:
# Impute the value for Energy Star Dishwasher for 2009 based on other years
# currently go with -9 as no data has been recorded for this
dfs[2009]['ESDISHW'] = -9

In [76]:
# Combine all categories of BTU usage into BTUXXOTH
# --- Electric
dfs[2001]['BTUELOTH'] = dfs[2001][["BTUELAPL","BTUELRFG","BTUELFZZ","BTUELCOL","BTUELDWH","BTUELCDR"]].sum(axis = 1)
dfs[2005]['BTUELOTH'] = dfs[2005][["BTUELAPL","BTUELRFG","BTUELFZZ","BTUELCOL","BTUELDWH","BTUELCDR"]].sum(axis = 1)
dfs[2015]['BTUELOTH'] = dfs[2015][["BTUELRFG1","BTUELRFG2","BTUELFRZ","BTUELCOK","BTUELMICRO","BTUELCW","BTUELCDR",\
                                "BTUELDWH","BTUELLGT","BTUELTVREL","BTUELTV1","BTUELTV2","BTUELAHUHEAT","BTUELAHUCOL",\
                                   "BTUELEVAPCOL","BTUELCFAN","BTUELDHUM","BTUELHUM","BTUELPLPMP","BTUELHTBPMP","BTUELHTBHEAT",\
                                   "BTUELNEC"]].sum(axis = 1)

# --- LPG
dfs[2001].rename(columns = {"BTULPAPL":"BTULPOTH"}, inplace = True)
dfs[2005].rename(columns = {"BTULPAPL":"BTULPOTH"}, inplace = True)
dfs[2015]['BTULPOTH'] = dfs[2015][["BTULPCOK","BTULPCDR","BTULPNEC"]].sum(axis = 1)

# --- Natural Gas
dfs[2001].rename(columns = {"BTUNGAPL":"BTUNGOTH"}, inplace = True)
dfs[2005].rename(columns = {"BTUNGAPL":"BTUNGOTH"}, inplace = True)
dfs[2015]['BTUNGOTH'] = dfs[2015][["BTUNGCOK","BTUNGCDR","BTUNGPLHEAT","BTUNGHTBHEAT","BTUNGNEC"]].sum(axis = 1)

# --- Fuel Oil
dfs[2001]['BTUFOOTH'] = dfs[2001][["BTUFOAPL","BTUKRAPL"]].sum(axis = 1)
dfs[2005].rename(columns = {"BTUFOAPL":"BTUFOOTH"}, inplace = True)
dfs[2015].rename(columns = {"BTUFONEC":"BTUFOOTH"}, inplace = True)

# for 2001, merge the total for BTU from FO and Kerosene into one col
dfs[2001]['BTUFOSPH'] = dfs[2001][["BTUFOSPH","BTUKRSPH"]].sum(axis = 1)
dfs[2001]['BTUFOWTH'] = dfs[2001][["BTUFOWTH","BTUKRWTH"]].sum(axis = 1)


E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [77]:
# Combine all categories of DOL usage into DOLXXOTH
# --- Electric
dfs[2001]['DOLELOTH'] = dfs[2001][["DOLELAPL","DOLELRFG","DOLELFZZ","DOLELCOL","DOLELDWH","DOLELCDR"]].sum(axis = 1)
dfs[2005]['DOLELOTH'] = dfs[2005][["DOLELAPL","DOLELRFG","DOLELFZZ","DOLELCOL","DOLELDWH","DOLELCDR"]].sum(axis = 1)
dfs[2015]['DOLELOTH'] = dfs[2015][["DOLELRFG1","DOLELRFG2","DOLELFRZ","DOLELCOK","DOLELMICRO","DOLELCW","DOLELCDR",\
                                   "DOLELDWH","DOLELLGT","DOLELTVREL","DOLELTV1","DOLELTV2","DOLELAHUHEAT","DOLELAHUCOL",\
                                   "DOLELCFAN","DOLELDHUM","DOLELHUM","DOLELPLPMP","DOLELHTBPMP","DOLELHTBHEAT",\
                                   "DOLELNEC"]].sum(axis = 1)

# --- LPG
dfs[2001].rename(columns = {"DOLLPAPL":"DOLLPOTH"}, inplace = True)
dfs[2005].rename(columns = {"DOLLPAPL":"DOLLPOTH"}, inplace = True)
dfs[2015]['DOLLPOTH'] = dfs[2015][["DOLLPCOK","DOLLPCDR","DOLLPNEC"]].sum(axis = 1)

# --- Natural Gas
dfs[2001].rename(columns = {"DOLNGAPL":"DOLNGOTH"}, inplace = True)
dfs[2005].rename(columns = {"DOLNGAPL":"DOLNGOTH"}, inplace = True)
dfs[2015]['DOLNGOTH'] = dfs[2015][["DOLNGCOK","DOLNGCDR","DOLNGPLHEAT","DOLNGHTBHEAT","DOLNGNEC"]].sum(axis = 1)

# --- Fuel Oil
dfs[2001]['DOLFOOTH'] = dfs[2001][["DOLFOAPL","DOLKRAPL"]].sum(axis = 1)
dfs[2005].rename(columns = {"DOLFOAPL":"DOLFOOTH"}, inplace = True)
dfs[2015].rename(columns = {"DOLFONEC":"DOLFOOTH"}, inplace = True)

# for 2001, merge the total for DOL from FO and Kerosene into one col
dfs[2001]['DOLFOSPH'] = dfs[2001][["DOLFOSPH","DOLKRSPH"]].sum(axis = 1)
dfs[2001]['DOLFOWTH'] = dfs[2001][["DOLFOWTH","DOLKRWTH"]].sum(axis = 1)



E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [ ]:
# Calculate the TOTAL BTU VALUES
# TOTALBTU = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
# TOTALBTUSPH = BTULPSPH + BTUNGSPH + BTUFOSPH + BTUELSPH
# TOTALBTUWTH = BTULPWTH + BTUNGWTH + BTUFOWTH + BTUELWTH
# TOTALBTUOTH = BTULPAPL + BTUNGOTH + BTUFOAPL + BTUELOTH + BTUELFRG +BTUELCOL
# TOTALDOLLAR = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
# TOTALDOLSPH = DOLLPSPH+ DOLNGSPH + DOLFOSPH + DOLELSPH
# TOTALDOLWTH = DOLLPWTH+ DOLNGWTH+ DOLFOWTH+ DOLELWTH
# TOTALDOLOTH = DOLLPOTH + DOLNGOTH+ DOLFOOTH + DOLELOTH + DOLELFRG + DOLELCOL

# --- Electric
for y in years:
    dfs[y]['']


Columns to be dropped in 2001
'SOLARAUX', 'FOILAUX', 'KEROAUX', "BTUELAPL","BTUELRFG","BTUELFZZ","BTUELCOL","BTUELDWH","BTUELCDR", "BTUFOAPL","BTUKRAPL", "BTUKRSPH", "BTUKRWTH", "DOLELAPL","DOLELRFG","DOLELFZZ","DOLELCOL","DOLELDWH","DOLELCDR", "DOLFOAPL","DOLKRAPL", "DOLKRSPH", "DOLKRWTH"

Columns to be dropped inn 2005
'SOLARAUX', 'FOILAUX', 'KEROAUX', "BTUELAPL","BTUELRFG","BTUELFZZ","BTUELCOL","BTUELDWH","BTUELCDR", "DOLELAPL","DOLELRFG","DOLELFZZ","DOLELCOL","DOLELDWH","DOLELCDR"


columns to  be dropped in 2009
"FURNFUEL","RADFUEL","PIPEFUEL","RMHTFUEL","HSFUEL","FPFUEL","RNGFUEL","DIFFUEL", "TOTSQFT_EN"

Columns to be dropped in 2015
"EQUIPAUXTYPE","FUELAUX", "TOTSQFT_EN", "BTUELRFG1","BTUELRFG2","BTUELFRZ","BTUELCOK","BTUELMICRO","BTUELCW","BTUELCDR","BTUELDWH","BTUELLGT","BTUELTVREL","BTUELTV1","BTUELTV2","BTUELAHUHEAT","BTUELAHUCOL","BTUELEVAPCOL","BTUELCFAN","BTUELDHUM","BTUELHUM","BTUELPLPMP","BTUELHTBPMP","BTUELHTBHEAT","BTUELNEC", "BTULPCOK","BTULPCDR","BTULPNEC", "BTUNGCOK","BTUNGCDR","BTUNGPLHEAT","BTUNGHTBHEAT","BTUNGNEC", "DOLELRFG1","DOLELRFG2","DOLELFRZ","DOLELCOK","DOLELMICRO","DOLELCW","DOLELCDR","DOLELDWH","DOLELLGT","DOLELTVREL","DOLELTV1","DOLELTV2","DOLELAHUHEAT","DOLELAHUCOL","DOLELCFAN","DOLELDHUM","DOLELHUM","DOLELPLPMP","DOLELHTBPMP","DOLELHTBHEAT","DOLELNEC", "DOLLPCOK","DOLLPCDR","DOLLPNEC", "DOLNGCOK","DOLNGCDR","DOLNGPLHEAT","DOLNGHTBHEAT","DOLNGNEC"

